# Update HYSETS Catchment Polygons

The HYSETS dataset {cite}`arsenault2020comprehensive` contains an "artificial boundaries" flag to indicate where the catchment boundary for the monitoring location is approximated due to either missing data or uncertainty in catchment delineation, in general due to small drainage area.  In July 2022, the Water Survey of Canada published updated polygons for over 8000 catchments in Canada.  We use updated catchment boundaries where available and additionally check the USGS database for updated polygons.  Polygons updated from USGS and WSC official sources cover 64% of the "artificial bounds" flagged catchments, and the remaining flags are updated using the closest matching station in the [BCUB dataset](https://doi.org/10.5683/SP3/JNKZVT). 

To start, download the HYSETS catchment polygons (`HYSETS_watershed_boundaries.zip`) from [that dataset's open data repository](https://osf.io/rpc3w/).

The resulting updated polygons are used in the next chapter/section to extract and validate attributes as part of the supporting information for technical validation of the associated publication.

## Import Data

### Import HYSETS data

In [ ]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from urllib.request import urlopen
import json

In [ ]:
# set the filename where the updated results will be stored
updated_fpath = 'data/BCUB_watershed_bounds_updated.geojson'

In [ ]:
# import the HYSETS attributes data
hysets_df = pd.read_csv('data/HYSETS_watershed_properties.txt', sep=';')
hysets_df.columns

### Import (BCUB) study region bounds

Get the region bounds from the BCUB dataset [https://doi.org/10.5683/SP3/JNKZVT](https://doi.org/10.5683/SP3/JNKZVT) or just skip this step and use the pre-processed file (`data/data/study_region_stations.geojson`).

In [ ]:
# import the BCUB (study) region boundary
region_gdf = gpd.read_file('data/BCUB_regions_4326.geojson')
region_gdf = region_gdf.to_crs(3005)
# simplify the geometries (100m threshold) and add a small buffer (250m) to capture HYSETS station points recorded with low accuracy near boundaries
region_gdf.geometry = region_gdf.simplify(100).buffer(500)
# region_gdf = region_gdf.to_crs(4326)

In [ ]:
# get the stations contained in the study region
centroids = hysets_df.apply(lambda x: Point(x['Centroid_Lon_deg_E'], x['Centroid_Lat_deg_N']), axis=1)
hysets_points = gpd.GeoDataFrame(hysets_df, geometry=centroids, crs='EPSG:4326')
hysets_points.to_crs(3005, inplace=True)
hysets_points.head(4)

Note that these are just the artificial bounds flagged rows, below we check for other corrections/updates from official sources.

### Load the original HYSETS polygons

In [ ]:
hs_path = 'data/catchment_polygons/HYSETS_watershed_boundaries/HYSETS_watershed_boundaries_20200730.shp'
hs_polygons = gpd.read_file(hs_path)
hs_polygons = hs_polygons.set_crs(4326)

In [ ]:
bcub_gdf = gpd.read_file('data/study_region_stations.geojson')
print(bcub_gdf.crs)
# the bcub geometries are (centroid) points from the HYSETS properties, 
# set the geometry to the HYSETS polygons instead
catchment_geometries = bcub_gdf.apply(lambda x: hs_polygons.loc[hs_polygons['OfficialID'] == x['Official_ID'], 'geometry'].values[0], axis=1)
bcub_gdf['geometry'] = catchment_geometries
bcub_gdf.to_crs(3005, inplace=True)
hs_polygons.to_crs(3005, inplace=True)

In [ ]:
ab_df = bcub_gdf.loc[bcub_gdf['Flag_Artificial_Boundaries']== 1, :].copy()
print(f'{len(ab_df)}/{len(bcub_gdf)} catchment geometries are flagged "artificial bounds"')

In [ ]:
wsc_ab = ab_df.loc[ab_df['Source'] == 'HYDAT', :].copy()
usgs_ab = ab_df.loc[ab_df['Source'] == 'USGS', :].copy()
print(f'{len(wsc_ab)}/{len(usgs_ab)} WSC/USGS artificial bounds flags ')

### Check for updated WSC polygons

The updated WSC catchments can be accessed at the Environment and Climate Change Canada (ECCC) [hydrometrics page](https://collaboration.cmc.ec.gc.ca/cmc/hydrometrics/www/HydrometricNetworkBasinPolygons/).  We only need to download the region files associated with the study region, corresponding to the first two digits of the station identifier (official id).

In [ ]:
wsc_stn_df = bcub_gdf.loc[bcub_gdf['Source'] == 'HYDAT'].copy()
wsc_stns = wsc_stn_df['Official_ID']
prefixes = list(set([e[:2] for e in wsc_stns]))
prefixes

Download the above files `<2-digit identifier>.zip` and extract them in the `data` folder.

In [ ]:
# find updated catchment polygons
def retrieve_and_update_WSC_polygon(stn_id):
    """
    Returns an updated WSC polygon if it exists or returns an empty (geo)dataframe.
    """
    stn_id = row['Official_ID']
    stn_prefix = stn_id[:2]
    catchment_path = f'data/catchment_polygons/{stn_prefix}/{stn_id}/{stn_id}_DrainageBasin_BassinDeDrainage.shp'
    if os.path.exists(catchment_path):
        print(f'Updated polygon found for {stn_id}')
        updated_polygon = gpd.read_file(catchment_path)
        updated_polygon.to_crs(3005, inplace=True)
        return updated_polygon
    return gpd.GeoDataFrame()

def retrieve_and_update_WSC_station_location(stn_id):
    """
    Returns an updated WSC station location if it exists or returns an empty (geo)dataframe.
    """
    stn_id = row['Official_ID']
    stn_prefix = stn_id[:2]
    file_path = f'data/catchment_polygons/{stn_prefix}/{stn_id}/{stn_id}_Station.shp'
    if os.path.exists(file_path):
        print(f'Updated station location found for {stn_id}')
        updated_pt = gpd.read_file(file_path)
        updated_pt.to_crs(3005, inplace=True)
        return updated_pt
    return gpd.GeoDataFrame()

### Check for updated USGS polygons

In [ ]:
# api url for nwis sites
usgs_api_url = 'https://labs.waterdata.usgs.gov/api/nldi/linked-data/nwissite/'

In [ ]:
def retrieve_usgs_stn_data(stn):
    # query the NWIS with the station number to get the station coordinates    
    try:
        query_url = usgs_api_url + f'USGS-{stn}'
        usgs_data = pd.read_json(query_url)
        usgs_stn_loc = usgs_data['features'][0]['geometry']['coordinates']
        stn_pt = Point(*usgs_stn_loc)
        return gpd.GeoDataFrame(geometry=[stn_pt], crs=4326)
    except Exception as ex:
        msg = f'USGS station query failed for {stn}. {ex}'
        print(msg)
        return pd.DataFrame()

In [ ]:
def usgs_basin_polygon_query(url):
    """USGS polygons are in EPSG:4326 crs"""
    response = urlopen(url)
    json_data = response.read().decode('utf-8', 'replace')
    d = json.loads(json_data)
    return gpd.GeoDataFrame.from_features(d['features'], crs='EPSG:4326')

In [ ]:
def retrieve_usgs_basin_data(stn):
    """Retrieve the USGS basin polygon and station location from the NLDI API. 
    If there is no basin for the station, use the NLDI to retrieve upstream 
    and downstream boundaries.  
    Pick the one closest in (HYSETS published) area to the station location."""    
    
    # query the basin polygon from USGS
    basin_query = usgs_api_url + f'USGS-{stn}/basin?simplified=false&splitCatchment=false'    
    try:
        usgs_basin_df = usgs_basin_polygon_query(basin_query)
        # dissolve the basin polygons
        usgs_basin_df = usgs_basin_df.dissolve()
        usgs_basin_df = usgs_basin_df.to_crs(3005)
        # check if geometry is multipolygon
        if usgs_basin_df.geometry.type.values[0] == 'MultiPolygon':
            print(f'   ...MultiPolygon detected, attemping to make geometry valid.')
            usgs_basin_df = usgs_basin_df.explode()
            usgs_basin_df['area'] = usgs_basin_df.geometry.area / 1E6
            usgs_basin_df['area_pct'] = usgs_basin_df['area'] / usgs_basin_df['area'].sum()
            usgs_basin_df = usgs_basin_df[usgs_basin_df['area_pct'] > 0.95]
            if len(usgs_basin_df) > 1:
                raise Exception('USGS basin polygon query returned multiple polygons.')

        return usgs_basin_df
    except Exception as ex:
        print(f'USGS basin polygon query failed for {stn}.  {ex}')
        return pd.DataFrame()

In [ ]:
bcub_gdf['geometry_updated'] = False
# set a variable to prevent jupyter book from processing
# during book building step
process_step = False
if process_step:
    for i, row in bcub_gdf.iterrows():
        stn_id = row['Official_ID']
        source = row['Source']
        if source == 'HYDAT':
            # pt = retrieve_and_update_WSC_station_location(stn_id)
            polygon = retrieve_and_update_WSC_polygon(stn_id)            
        elif source == 'USGS':
            # pt = retrieve_usgs_stn_data(stn_id)
            polygon = retrieve_usgs_basin_data(stn_id)    
        if not polygon.empty:
            assert polygon.crs == 'EPSG:3005'
            pt = polygon.geometry.centroid
            pt = pt.to_crs(4326)
            lat, lon = pt.geometry.x[0], pt.geometry.y[0]
    
            bcub_gdf.loc[i, 'Centroid_Lat_deg_N'] = lat
            bcub_gdf.loc[i, 'Centroid_Lon_deg_E'] = lon
            bcub_gdf.loc[i, 'geometry'] = polygon.geometry.values[0]
            bcub_gdf.loc[i, 'geometry_updated'] = True
            bcub_gdf.loc[i, 'Flag_Artificial_Boundaries'] = False    
        else:
            polygon = hs_polygons.loc[hs_polygons['OfficialID'] == stn_id, 'geometry'].copy()
            bcub_gdf.loc[i, 'geometry'] = polygon.geometry.values[0]

In [ ]:
ab_df_remaining = bcub_gdf.loc[bcub_gdf['Flag_Artificial_Boundaries'] == 1, :].copy()
print(f'{len(ab_df_remaining)}/{len(bcub_gdf)} catchment geometries remain flagged "artificial bounds" ({len(ab_df) - len(ab_df_remaining)} updated.)')

### For the remaining 'artificial bounds' catchments, derive them from USGS 3DEP DEM.

For all of the remaining rows labeled "FLAG_Artificial_Boundaries", follow the methodology in the [BCUB dataset demo](https://dankovacek.github.io/bcub_demo/notebooks/2_DEM_Preprocessing.html) and reprocess the catchment bounds.

First, find which regions have flagged geometries:

In [ ]:
# check to ensure all geometries are polygon type
assert all(ab_df_remaining.geometry.geom_type == 'Polygon')
if 'index_right' in ab_df_remaining.columns:
    ab_df_remaining.drop(columns=['index_right'], axis=1, inplace=True)

In [ ]:
if process_step:
    ab_df_remaining['bcub_region_code'] = None
    for i, row in ab_df_remaining.iterrows():
        # region_polygon = region_gdf.loc[i, 'geometry']
        listed_da = row['Drainage_Area_km2']
        centroid = row['geometry'].centroid
        stn_id = row['Official_ID']
            
        # get the intersecting region
        intersecting_region = gpd.sjoin(ab_df_remaining.loc[[i]], region_gdf, how='inner', predicate='intersects')
        rc = intersecting_region['region_code_left']
        if rc.empty:
            foo = gpd.GeoDataFrame([stn_id], geometry=[centroid], crs='EPSG:3005')
            foo.to_file(f'data/catchment_polygons/{stn_id}_centroid.geojson', driver='GeoJSON')
            raise Exception('region search failed')
        else:
            ab_df_remaining.loc[i, 'bcub_region_code'] = intersecting_region['region_code_left'].values[0]


### Set DEM folder -- need processed stream raster

In [ ]:
dem_folder = '/home/danbot2/code_5820/large_sample_hydrology/bcub/processed_data/processed_dem/'

In [ ]:
import rasterio as rio
import rioxarray as rxr
from rasterio.features import dataset_features
from shapely.geometry import Point
from scipy.spatial import cKDTree

def retrieve_raster(filename):
    """
    Take in a file name and return the raster data, 
    the coordinate reference system, and the affine transform.
    """
    raster = rxr.open_rasterio(filename, mask_and_scale=True)
    crs = raster.rio.crs
    affine = raster.rio.transform(recalc=False)
    return raster, crs.to_epsg(), affine

def nearest_stream_pixel(dem_filepath, points_gdf):
    # Read the DEM raster
    with rio.open(dem_filepath) as src:
        dem_data = src.read(1)  # Read the first band
        transform = src.transform
        no_data = src.nodata
        
        # Get the coordinates of the stream pixels (assuming stream pixels are not no_data)
        stream_indices = np.column_stack(np.where(dem_data != no_data))
        
        # Convert raster indices to centroids of the pixels
        stream_coords = []
        for row, col in stream_indices:
            x_center, y_center = rio.transform.xy(transform, row, col, offset='center')
            stream_coords.append((x_center, y_center))

        # Build a KDTree for fast nearest-neighbor lookup
        stream_tree = cKDTree(stream_coords)
    
    # Function to find nearest stream pixel for a given point
    def find_nearest_stream(point):
        point_coords = np.array(point.coords[0])
        distance, idx = stream_tree.query(point_coords)
        nearest_stream_coord = stream_coords[idx]
        return nearest_stream_coord, distance
    
    # Apply the function to each point in the GeoDataFrame
    results = points_gdf.geometry.apply(find_nearest_stream)
    
    # Create a new GeoDataFrame with the results
    nearest_coords = [result[0] for result in results]
    distances = [result[1] for result in results]
    nearest_points_gdf = gpd.GeoDataFrame({
        'geometry': [Point(coord) for coord in nearest_coords],
        'distance_to_stream': distances,
        'Official_ID': points_gdf['Official_ID']
    }, crs=points_gdf.crs)
    
    return nearest_points_gdf    

### Find the nearest pixel in the stream raster corresponding to the point geometry

Note that where the catchment geometry is missing in HYSETS, the centroid point is just the station location, and the geometry is simply a square centred at the station location and with an area equal to the drainage area reported by the official source {cite}`arsenault2020comprehensive`.  We then find the nearest stream pixel to the "centroid" which is not actually the centroid but the reported station location.

In [ ]:
ab_df_remaining = ab_df_remaining.sort_values(by=['bcub_region_code'])

pour_pt_filenames = []
for rc in list(set(ab_df_remaining['bcub_region_code'])):
    print(f'Processing {rc}')
    to_check = ab_df_remaining[ab_df_remaining['bcub_region_code'] == rc].copy()[['Official_ID', 'Centroid_Lat_deg_N', 'Centroid_Lon_deg_E']]
    to_check['geometry'] = to_check.apply(lambda x: Point(x['Centroid_Lon_deg_E'], x['Centroid_Lat_deg_N']), axis=1)
    check_pts = gpd.GeoDataFrame(to_check, crs='4326')
    check_pts = check_pts.to_crs(3005)
    # load the stream raster
    stream_dem_path = os.path.join(dem_folder, f'{rc}_USGS_3DEP_3005_stream.tif')
    temp_fpath = f'data/catchment_polygons/updated_polygon_set/{rc}_pour_points.shp'
    if not os.path.exists(temp_fpath):
        nearest_px = nearest_stream_pixel(stream_dem_path, check_pts)
        nearest_px.to_file(temp_fpath)
    print(f'   ...processed {rc}')
    pour_pt_filenames.append(temp_fpath)  


### Delineate catchments with Whiteboxtools

In [ ]:
import whitebox 
wbt = whitebox.WhiteboxTools()
wbt.verbose = False

In [ ]:
if process_step:
    for f in pour_pt_filenames:
        rc = f.split('/')[-1].split('_')[0]
        print(f'   ...processing {rc} pour points')
        d8_path = f'/home/danbot2/code_5820/large_sample_hydrology/bcub/processed_data/processed_dem/{rc}_USGS_3DEP_3005_fdir.tif'
        assert os.path.exists(d8_path)
        assert os.path.exists(f)
        output_fpath = os.path.join(os.getcwd(), f.replace('_pour_points.shp', '_basins.tif'))
        if not os.path.exists(output_fpath):
            wbt.unnest_basins(
                d8_path, 
                os.path.join(os.getcwd(), f), 
                output_fpath, 
                esri_pntr=False, 
            )
        print(f'   ...completed processing of {rc}')

### Convert raster to vector polygons

In [ ]:
def filter_and_explode_geoms(gdf, min_area):
    gdf.geometry = gdf.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)    
    gdf = gdf.explode(index_parts=True)
    gdf['area'] = gdf.geometry.area / 1E6
    gdf = gdf[gdf['area'] >= min_area * 0.95]    
    return gdf

    
def raster_to_vector_batch(rc, raster_fname, raster_crs, resolution, min_area, temp_folder):
    """
    If we send too many pour points per batch to the Whitebox "unnest" function, 
    we generate a huge number of temporary raster files that could easily 
    exceed current SSD disk capacities.
    """
    raster_path = os.path.join(temp_folder, raster_fname)
    raster_no = int(raster_fname.split('_')[-1].split('.')[0])
    
    polygon_path = os.path.join(temp_folder, f'{rc}_temp_polygons_{raster_no:05}.shp')
    
    output_path = polygon_path.replace('.shp', '.geojson')
    if os.path.exists(output_path):
        print(f'{output_path.split("/")[-1]} exists')
        existing_df = gpd.read_file(output_path)
        return existing_df
    else:
        # this function creates rasters of ordered 
        # sets of non-overlapping basins
        wbt.raster_to_vector_polygons(
            raster_path,
            polygon_path,
        )
    
        gdf = gpd.read_file(polygon_path, crs=raster_crs)
    
        # simplify the polygon geometry to avoid self-intersecting polygons
        simplify_dim = 0.5 * np.sqrt(resolution[0]**2 + resolution[1]**2)
        simplify_dim = abs(resolution[0])
        buffer_dim = 10
        gdf.geometry = gdf.geometry.buffer(buffer_dim)
        gdf.geometry = gdf.geometry.simplify(simplify_dim)
        gdf.geometry = gdf.geometry.buffer(-1.0 * buffer_dim)
        
        gdf = filter_and_explode_geoms(gdf, min_area)
    
        assert (gdf.geometry.geom_type == 'Polygon').all()
            
        gdf.drop(labels=['area'], inplace=True, axis=1)
        gdf.to_file(output_path)    
        return gdf
    

def format_batch_geometries(all_polygons, ppt_batch, region_raster_crs):
    
    ppt_batch['ppt_x'] = ppt_batch.geometry.x.astype(int)
    ppt_batch['ppt_y'] = ppt_batch.geometry.y.astype(int)
    ppt_batch.drop(inplace=True, labels=['geometry'], axis=1)

    # concatenate the polygon batches
    for f in all_polygons:
        print(len(f))

    batch_polygons = gpd.GeoDataFrame(pd.concat(all_polygons), crs=region_raster_crs)
    print(len(all_polygons))
    batch_polygons.sort_values(by='VALUE', inplace=True)
    batch_polygons.reset_index(inplace=True, drop=True)
    # print(batch_polygons)
    # print(asdf)
    
    batch = gpd.GeoDataFrame(pd.concat([batch_polygons, ppt_batch], axis=1), crs=region_raster_crs)
    # print(batch)
    # print(asdf)
    batch['centroid_x'] = batch.geometry.centroid.x.astype(int)
    batch['centroid_y'] = batch.geometry.centroid.y.astype(int)
    return batch  

In [ ]:
min_area = 1.0 # km^2

if process_step:
    for f in pour_pt_filenames:
        rc = f.split('/')[-1].split('_')[0]
        output_fpath = os.path.join(os.getcwd(), f'data/catchment_polygons/{rc}_basins_BCUB.geojson')
        catchment_folder = 'data/catchment_polygons/updated_polygon_set'
        temp_folder = os.path.join(os.getcwd(), catchment_folder, 'temp')
        # batch_output_fpath = output_fpath.replace('.geojson', f'_final.geojson')
        
        if not os.path.exists(temp_folder):
            os.makedirs(temp_folder)
    
        ppt_batch = gpd.read_file(f)
        print(f'{len(ppt_batch)} ppts in pour point batch')
        print('')
            
        basin_raster_files = [e for e in os.listdir(catchment_folder) if e.startswith(f'{rc}_basins')]
        print(f'   ...processing {rc} basin rasters to vector.')
        catchment_batch = []
        for bf in sorted(basin_raster_files):
            print(f'processing {bf}')
            raster_fpath = os.path.join(os.getcwd(), catchment_folder, bf)
            assert os.path.exists(raster_fpath)
            raster, raster_crs, _ = retrieve_raster(raster_fpath)
            resolution = raster.rio.resolution()
            
            cdf = raster_to_vector_batch(rc, raster_fpath, raster_crs, resolution, min_area, temp_folder)
            
            catchment_batch.append(cdf)
    
        batch_df = format_batch_geometries(catchment_batch, ppt_batch, raster_crs)
        batch_df.to_file(output_fpath)
        

In [ ]:
updated_files = sorted([e for e in os.listdir('data/catchment_polygons') if e.endswith('basins_BCUB.geojson')])
all_dfs = []
if process_step:
    for f in updated_files:
        udf = gpd.read_file(os.path.join('data/catchment_polygons', f))
        udf['region_code'] = f.split('/')[-1].split('_')[0]
        
        for i, row in udf.iterrows():
            geom = row['geometry']
            
            stn_id = row['Official_I']
            if stn_id not in bcub_gdf['Official_ID'].values:
                raise Exception(f'   ...{stn_id} not found in hysets set')
            # update the geometries
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'geometry'] = geom
            pt = geom.centroid
            pt_reproj = gpd.GeoDataFrame(geometry=[pt], crs=3005).to_crs(4326)
            lat, lon = pt_reproj.geometry.x[0], pt_reproj.geometry.y[0]
            
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'Centroid_Lat_deg_N'] = lat
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'Centroid_Lon_deg_E'] = lon
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'geometry'] = geom
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'geometry_updated'] = True
            bcub_gdf.loc[bcub_gdf['Official_ID'] == stn_id, 'Flag_Artificial_Boundaries'] = False

    ab_df_remaining = bcub_gdf.loc[bcub_gdf['Flag_Artificial_Boundaries'] == 1, :].copy()
    print(f'{len(ab_df_remaining)}/{len(bcub_gdf)} catchment geometries remain flagged "artificial bounds" ({len(ab_df) - len(ab_df_remaining)} updated.)')

In [ ]:
# save updated_fpath
# bcub_gdf.to_file(updated_fpath)
print(f'saved updated polygon geometries to {updated_fpath}')

## Citations

```{bibliography}
:filter: docname in docnames
```